In [1]:
import sys
sys.path.append('../')

import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from src.database import MongoDB

%matplotlib inline

pd.options.display.max_columns = 100

flatten = lambda ll: [x for l in ll for x in l]

## Check data quality

In [2]:
db = MongoDB()
matches_data = db.query_all_data('matches_data')

rows = []
for doc in matches_data:
    temp = []
    temp.append(doc['match_id'])
    temp.append(doc['duration'])
    for player in doc['players']:
        temp.append(player['abandons'])
        temp.append(player['account_id'])
    rows.append(temp)

Querying data from matches_data.


In [3]:
df = pd.DataFrame(rows, columns=['match_id', 'duration'] + flatten([['abandons_' + str(i), 'player_' + str(i)] for i in range(10)]))
df.shape

(38357, 22)

In [4]:
df['radiant_abandons'] = df[['abandons_' + str(i) for i in range(5)]].sum(axis=1)
df['dire_abandons'] = df[['abandons_' + str(i) for i in range(5, 10)]].sum(axis=1)

df['no_radiant_identified'] = (~df[['player_' + str(i) for i in range(5)]].isna() == True).\
    apply(lambda x: sum(x), axis=1)
df['no_dire_identified'] = (~df[['player_' + str(i) for i in range(5, 10)]].isna() == True).\
    apply(lambda x: sum(x), axis=1)

df.head()

,match_id,duration,abandons_0,player_0,abandons_1,player_1,abandons_2,player_2,abandons_3,player_3,abandons_4,player_4,abandons_5,player_5,abandons_6,player_6,abandons_7,player_7,abandons_8,player_8,abandons_9,player_9,radiant_abandons,dire_abandons,no_radiant_identified,no_dire_identified
0,4154000815,2470,0,NaN,0,NaN,0,299711286.0,0,NaN,0,334260386.0,0,135550506.0,0,100664395.0,0,397697319.0,0,198316694.0,0,174185760.0,0,0,2,5
1,4152678603,2239,0,NaN,0,NaN,0,NaN,0,200445858.0,1,144286464.0,0,NaN,0,NaN,0,254093217.0,0,NaN,0,NaN,1,0,2,1
2,4152494309,3286,0,212752913.0,0,257710284.0,0,NaN,0,NaN,0,NaN,0,52058001.0,0,NaN,0,94871311.0,0,125012778.0,0,157017119.0,0,0,2,4
3,4152425012,2408,0,NaN,0,104438886.0,0,NaN,0,NaN,0,NaN,0,198782054.0,0,191886982.0,0,143928825.0,0,115340026.0,0,NaN,0,0,1,4
4,4152417604,2192,0,131116276.0,0,361645365.0,0,120410884.0,0,887383170.0,0,119571905.0,0,NaN,0,NaN,0,370089153.0,0,NaN,0,150848809.0,0,0,5,2


In [5]:
df[(df.radiant_abandons != 0) | (df.dire_abandons != 0)].shape

(2522, 26)

In [6]:
df[(df.no_radiant_identified > 3) & (df.no_dire_identified > 3)].shape

(16522, 26)

In [7]:
df[(df.no_radiant_identified > 4) & (df.no_dire_identified > 4)].shape

(3787, 26)